In [3]:
import requests
import numpy as np

from sqlalchemy.orm import sessionmaker
from scipy import interpolate
from opticalsimulation.settings import Engine
from opticalsimulation.models import Material, OpticalIndex

Session = sessionmaker(bind=Engine)

## filmetricsからデータを追加

In [4]:
def add_opticalindex(url):
    response = requests.get(url)
    if response.status_code == 200:
        
        name = url.split('/')[-1]
        text = response.text
        
        session = Session()

        material = session.query(Material).filter_by(name=name).one_or_none()
        if material is None:
            material = Material(name=name)
            session.add(material)
            material = session.query(Material).filter_by(name=name).one()
            lines = text.splitlines()
            lines.pop(0)
            opticalindexes = [OpticalIndex(material_id=material.id,wavelength=line.split()[0],
                                           n=line.split()[1],k=line.split()[2]) for line in lines]
            session.add_all(opticalindexes)
            session.commit()
        session.close()
    return response.status_code



## データ出力

In [5]:
def get_material_list():
    session = Session()
    qs = session.query(Material).all()
    session.close()
    return [(q.id,q.name) for q in qs]

In [6]:
def get_material_name(id):
    session = Session()
    q = session.query(Material).filter_by(id=id).one_or_none()
    return q.name

In [7]:
def get_opticalindex(id):
    session = Session()
    qs = session.query(OpticalIndex).filter_by(material_id=id).order_by(OpticalIndex.wavelength).all()
    session.close()
    ws = [q.wavelength for q in qs]
    nks = [complex(q.n,-q.k) for q in qs]
    return [ws,nks]

In [8]:
def fitted_opticalindex(id,start,end,step):    
    session = Session()
    qs = session.query(OpticalIndex).filter_by(material_id=id).order_by(OpticalIndex.wavelength).all()
    session.close()
    if len(qs) > 1:
        ws = [q.wavelength for q in qs]
        if ws[0] <= start and ws[-1] >= end:
            ns = [q.n for q in qs]
            ks = [q.k for q in qs]
            n_fitted = interpolate.interp1d(ws,ns)
            k_fitted = interpolate.interp1d(ws,ks)
            x = np.arange(start,end,step)
            return [complex(n,-k) for n,k in zip(n_fitted(x),k_fitted(x))]
    return []

## データ削除

In [9]:
def delete_opticalindex(id):
    Session = sessionmaker(bind=Engine)
    session = Session()
    q = session.query(Material).filter_by(id=id).one_or_none()
    if q is not None:
        session.query(OpticalIndex).filter_by(material_id=q.id).delete()
        session.delete(q)
        session.commit()
    session.close()


In [8]:
# Test
#url = 'https://www.filmetricsinc.jp/technology/refractive-index-database/download/Acrylic'
#url = 'https://www.filmetricsinc.jp/technology/refractive-index-database/download/Al10Ga90As'
url = 'https://www.filmetricsinc.jp/technology/refractive-index-database/download/Al20Ga80As'

add_opticalindex(url)


200

In [10]:
get_material_list()

[(1, 'Al10Ga90As'), (2, 'Al20Ga80As')]

In [11]:
get_material_name(1)

'Al10Ga90As'

In [12]:
fitted_opticalindex(1,300,500,20)

[(3.7316666666666665-2.013333333333333j),
 (3.55875-1.921j),
 (3.5425376344086024-1.9629139784946237j),
 (3.649778846153846-2.0683653846153844j),
 (4.003641025641025-2.19725641025641j),
 (4.46-1.949j),
 (4.898909090909091-1.503342657342658j),
 (4.771058823529411-0.831803921568627j),
 (4.5114374999999995-0.5705056818181818j),
 (4.331413612565445-0.44901570680628267j)]

In [14]:
import opticalsimulation.database as db

In [17]:
url = 'https://www.filmetricsinc.jp/technology/refractive-index-database/download/Al2O3'
db.add_opticalindex(url)

200

In [18]:
db.fitted_opticalindex(1,300,400,10)

[(3.7316666666666665-2.013333333333333j),
 (3.618-1.937j),
 (3.55875-1.921j),
 (3.5348977272727273-1.9328295454545454j),
 (3.5425376344086024-1.9629139784946237j),
 (3.5797171717171716-2.007848484848485j),
 (3.649778846153846-2.0683653846153844j),
 (3.773081081081081-2.14918018018018j),
 (4.003641025641025-2.19725641025641j),
 (4.2944-2.1393999999999997j)]